In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
data=pd.read_csv('/content/coursera_course_dataset_v3.csv')

In [ ]:
final_data = data[['Title','Skills']]
final_data.head()

,Title,Skills
0,Google Cybersecurity,"Network Security, Python Programming, Linux, ..."
1,Google Data Analytics,"Data Analysis, R Programming, SQL, Business C..."
2,Google Project Management:,"Project Management, Strategy and Operations, ..."
3,IBM Data Science,"Python Programming, Data Science, Machine Lea..."
4,Google Digital Marketing & E-commerce,"Digital Marketing, Marketing, Marketing Manag..."


In [ ]:
# Assuming 'final_data' is your DataFrame containing the 'Skills' column
unique_skills = final_data['Skills'].unique()
num_unique_skills = len(unique_skills)
print("Number of unique skills:", num_unique_skills)


Number of unique skills: 601


In [ ]:
def preprocess_title(title):
    # Tokenization
    tokens = word_tokenize(title)
    # Lowercasing and removing punctuation
    tokens = [word.lower() for word in tokens if word.isalpha()]
    # Stemming
    stemmer = PorterStemmer()
    stemmed_tokens = [stemmer.stem(word) for word in tokens]
    # Joining tokens back into a string
    preprocessed_title = ' '.join(stemmed_tokens)
    return preprocessed_title

In [ ]:
final_data['Title'] = final_data['Title'].apply(preprocess_title)

<ipython-input-23-79748e1d996c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_data['Title'] = final_data['Title'].apply(preprocess_title)


In [ ]:
# TF-IDF Vectorization
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(final_data['Skills'])
y = final_data['Title']

In [ ]:
final_data["Skills"].str.len().value_counts()
# final_data.iloc[16,]

Skills
16     9
39     7
60     7
56     7
321    5
      ..
120    1
53     1
382    1
447    1
223    1
Name: count, Length: 348, dtype: int64

In [ ]:
final_data['Skills'] = final_data['Skills'].apply(lambda x: 'empty' if len(x) == 0 else x)

<ipython-input-24-bbfebaa1009e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_data['Skills'] = final_data['Skills'].apply(lambda x: 'empty' if len(x) == 0 else x)


In [ ]:
final_data["Skills"].str.len().value_counts()


Skills
16     9
39     7
60     7
56     7
321    5
      ..
120    1
53     1
382    1
447    1
223    1
Name: count, Length: 348, dtype: int64

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfv=TfidfVectorizer()

In [ ]:
tfv_matrix=tfv.fit_transform(final_data['Skills'])
tfv_matrix

<623x314 sparse matrix of type '<class 'numpy.float64'>'
	with 9971 stored elements in Compressed Sparse Row format>

In [ ]:
tfv_matrix.shape

(623, 314)

In [ ]:
from sklearn.metrics.pairwise import sigmoid_kernel
sig=sigmoid_kernel(tfv_matrix,tfv_matrix)

In [ ]:
len(final_data["Title"].unique())

618

In [ ]:
final_data["Title"]

0                    googl cybersecur
1                   googl data analyt
2                 googl project manag
3                     ibm data scienc
4                  googl digit market
                    ...              
618              googl workspac secur
619    cybersecur for tech profession
620           cybersecur in the cloud
621                appli cryptographi
622           python for activ defens
Name: Title, Length: 623, dtype: object

In [ ]:
indices = pd.Series(final_data.index,index = final_data["Title"]).drop_duplicates()

In [ ]:
indices

Title
googl cybersecur                    0
googl data analyt                   1
googl project manag                 2
ibm data scienc                     3
googl digit market                  4
                                 ... 
googl workspac secur              618
cybersecur for tech profession    619
cybersecur in the cloud           620
appli cryptographi                621
python for activ defens           622
Length: 623, dtype: int64

In [ ]:
def give_recommendation(title,sig=sig):
  idx=indices[title]
  sig_scores=list(enumerate(sig[idx]))
  sig_scores=sorted(sig_scores,key=lambda x:x[1],reverse=True)
  sig_scores=sig_scores[1:11]
  job_indices=[i[0] for i in sig_scores]
  return final_data[['Title','Skills']].iloc[job_indices]


In [ ]:
def recommender(title):
  job = final_data[final_data["Title"].str.contains(title)].iloc[0]["Title"]
  return give_recommendation(job)

In [ ]:
#final_data[final_data["Title"].str.contains('Cybersecurity')].iloc[0]["Title"]

'Google Cybersecurity'

In [ ]:
recommender('data scienc')

,Title,Skills
12,introduct to data scienc,"Data Science, Python Programming, Data Analys..."
5,ibm data analyst,"Python Programming, Microsoft Excel, Data Vis..."
19,appli data scienc,"Python Programming, Data Analysis, Machine Le..."
22,ibm data analyt with excel and r,"Data Analysis, R Programming, Computer Progra..."
297,from data to insight with googl cloud,"Google Cloud Platform, Cloud Computing, SQL, ..."
13,data scienc fundament with python and sql,"Python Programming, Data Analysis, General St..."
159,data scienc,"R Programming, Data Analysis, Statistical Pro..."
264,invest manag with python and machin learn,"Investment Management, Finance, Python Progra..."
14,ibm data engin,"Data Management, Databases, Database Administ..."
136,what is data scienc,"Data Science, Data Mining, Big Data, Data Ana..."


In [ ]:
# knn
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors

# Load the dataset
data = pd.read_csv('/content/coursera_course_dataset_v2_no_null.csv')

# Extract relevant columns
final_data = data[['Title','Skills']]

# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(final_data['Skills'])

# Compute cosine similarity matrix
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Fit Nearest Neighbors model
knn = NearestNeighbors(metric='cosine', algorithm='brute')
knn.fit(tfidf_matrix)

# Function to recommend jobs based on input title
def recommender(title, k=10):
    title_index = final_data[final_data["Title"].str.contains(title)].index[0]
    distances, indices = knn.kneighbors(tfidf_matrix[title_index], n_neighbors=k+1)
    recommended_indices = indices.flatten()[1:]
    return final_data.iloc[recommended_indices]



In [ ]:
recommender('Data Science')

,Title,Skills
12,Introduction to Data Science,"Data Science, Python Programming, Data Analys..."
5,IBM Data Analyst,"Python Programming, Microsoft Excel, Data Vis..."
19,Applied Data Science,"Python Programming, Data Analysis, Machine Le..."
22,IBM Data Analytics with Excel and R,"Data Analysis, R Programming, Computer Progra..."
297,From Data to Insights with Google Cloud,"Google Cloud Platform, Cloud Computing, SQL, ..."
13,Data Science Fundamentals with Python and SQL,"Python Programming, Data Analysis, General St..."
159,Data Science,"R Programming, Data Analysis, Statistical Pro..."
264,Investment Management with Python and Machine ...,"Investment Management, Finance, Python Progra..."
14,IBM Data Engineering,"Data Management, Databases, Database Administ..."
136,What is Data Science?,"Data Science, Data Mining, Big Data, Data Ana..."


In [ ]:
## k means

from sklearn.cluster import KMeans

##
final_data = data[['Title','Skills']]

# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(final_data['Skills'])

# Fit KMeans clustering model
num_clusters = 10  # You can adjust this value based on the dataset
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(tfidf_matrix)

# Assign clusters to each course
final_data['Cluster'] = kmeans.labels_

# Function to recommend jobs based on input title
def recommender(title):
    title_skills = tfidf_vectorizer.transform([final_data.loc[final_data['Title'] == title, 'Skills'].values[0]])
    title_cluster = kmeans.predict(title_skills)[0]
    similar_courses = final_data[final_data['Cluster'] == title_cluster]
    return similar_courses.head(10)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
<ipython-input-39-2483d5cbedcd>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_data['Cluster'] = kmeans.labels_


In [ ]:
recommender('Data Science')

,Title,Skills,Cluster
1,Google Data Analytics,"Data Analysis, R Programming, SQL, Business C...",0
3,IBM Data Science,"Python Programming, Data Science, Machine Lea...",0
5,IBM Data Analyst,"Python Programming, Microsoft Excel, Data Vis...",0
12,Introduction to Data Science,"Data Science, Python Programming, Data Analys...",0
14,IBM Data Engineering,"Data Management, Databases, Database Administ...",0
18,Data Engineering Foundations,"Data Management, Python Programming, Data Str...",0
22,IBM Data Analytics with Excel and R,"Data Analysis, R Programming, Computer Progra...",0
23,Data Analysis and Visualization Foundations,"Microsoft Excel, Data Visualization, Spreadsh...",0
33,Business Intelligence (BI) Analyst,"Data Management, Databases, SQL, Data Visuali...",0
37,Microsoft Power BI Data Analyst,"Data Analysis, Data Management, Data Mining, ...",0


In [ ]:
!pip install requests
!pip install bs4
!pip install html5lib

In [ ]:
## import requests
# URL = "https://www.naukri.com/fresher-jobs?src=gnbjobs_homepage_srch"
# r = requests.get(URL)
# #print(r.content)
import requests
from bs4 import BeautifulSoup

URL = "https://www.amazon.in/s?k=phones&i=electronics&crid=2YM7MZFLJUXM1&sprefix=phone%2Celectronics%2C200&ref=nb_sb_noss_1"
r = requests.get(URL)

soup = BeautifulSoup(r.text, 'html.parser')
#print(soup.prettify())

skills=[]
job_titles=[]
skill=soup.find('span',class_="a-size-medium a-color-base a-text-normal")
skills.append(skill)
print(skill)
#for page in pages:


None
